Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = ""

---

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [2]:
NAME = ""
COLLABORATORS = ""

---

In [ ]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx

In [4]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Helper functions

In [5]:
def isDistrictContiguous(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return nx.is_connected(district_graph)

In [6]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [7]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).geom_type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).geom_type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))      
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [8]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area 
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

# This Notebook will help you get started on NJ
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).

### This is the current assignment of precinct to congressional districts (12 of them for NJ)
Note that the map shown in DRA is slightly different. This is because some precincts are split in the real assignment, and some additional precinct are created to handle special situations such as prisoners and overseas citizens. You can ignore this for the class project and just use the data and functions provided.

In [ ]:
nj_current_assignment = pd.read_csv('Map_Data/precinct-assignments-congress-nj.csv')
nj_current_assignment

### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [ ]:
nj_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [11]:
shpfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shp'
dbffile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.dbf'
shxfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nj_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nj_precinct_boundaries[rec[3]]=geom

### This is the precinct boundary data 

This use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nj.csv. 

In [12]:
nj_contiguity = pd.read_csv('Contiguity_nj.csv', header=None)

In [ ]:
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(i, nj_current_assignment, nj_contiguity)))

In [ ]:
#Compactness of the current assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    

In [ ]:
# District Population of the current assignment
print(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12))

# A simple geographical  redistricting strategy

We can create a simple geopgraphical map, like we did for NH. In this case, we have 12 districts, so let's splitting the district in half North/South, and in 6th  East/West. 
New Hampshire's bounding box is (-75.559614,38.928519,-73.893979,41.357423) (https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/)
So let's start by splitting the state approximately though the middle longitude (-74.72) : everything west of longitude -71.583934 is in odd Districts, everything east is in even Districts. We will use the precinct centroids to assign them. Then we will divide each half per latitude on the ranges  (38.92, 39.3, 39.7, 40.1, 40.5,40.9,41.35)
Import the Map to DRA to look at it.

In [17]:
nj_longlat_assignment = nj_current_assignment.copy()
nj_longlat_assignment['District'] = 0
for index, row in nj_longlat_assignment.iterrows():
    try:
        if shape(nj_precinct_boundaries[row['GEOID20']]).geom_type == 'Polygon':
            centroid = Polygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nj_precinct_boundaries[row['GEOID20']]).geom_type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nj_precinct_boundaries[row['GEOID20']]).geom_type)
            pass
        if centroid.x <= -74.72:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 1
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 3
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 5
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 7
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 9
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 11
        else:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 2
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 4
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 6
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 8
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 10
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 12
    except KeyError: 
        pass
#print(nh_longitude_assignment)
nj_longlat_assignment.to_csv('Recitation maps/nj_longlat_map.csv',index=False)

In [ ]:
#Compactness of this Longlat assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    

In [ ]:
# District Population of this longlat assignment
print(getDistrictPopulations(nj_longlat_assignment,nj_precinct_data, 12))


# Now create your own redistricting maps
Remember to check for contiguity, and to ensure that the population of the districts are balanced (which is not the case in the example above.)

## Fair Map Seeding Algorithm

This section implements the seeding algorithm for the fair map. The goal is to select 12 initial seed precincts (one per district) that are:
- Geographically spread out across the state
- Have population sizes initially close to the average precinct population
- Consider demographic trends and try to pair demographic groups 
- Not biased towards any political party 
- Located in areas that allow for compact district growth

- Since final population of each district should be around NJPopulation/12.
- The seed population of each district should be 10% of NJ Population/12


In [ ]:
def get_precinct_centroid(geoid, boundaries):
    """Get the centroid coordinates for a single precinct."""
    try:
        if shape(boundaries[geoid]).geom_type == 'Polygon':
            centroid = Polygon(shape(boundaries[geoid])).centroid
        elif shape(boundaries[geoid]).geom_type == 'MultiPolygon':
            centroid = MultiPolygon(shape(boundaries[geoid])).centroid
        else:
            return None
        return (centroid.x, centroid.y)  # (longitude, latitude)
    except (KeyError, Exception):
        return None

def get_district_centroid(district_id, assignment, boundaries):
    """Get the centroid coordinates for a district (all precincts in that district)."""
    district_precincts = assignment[assignment['District'] == district_id]['GEOID20'].tolist()
    precinct_centroids = []
    for geoid in district_precincts:
        centroid = get_precinct_centroid(geoid, boundaries)
        if centroid:
            precinct_centroids.append(centroid)
    
    if not precinct_centroids:
        return None
    
    # Calculate average centroid (simple approach)
    avg_lon = sum(c[0] for c in precinct_centroids) / len(precinct_centroids)
    avg_lat = sum(c[1] for c in precinct_centroids) / len(precinct_centroids)
    return (avg_lon, avg_lat)

def get_precinct_neighbors(geoid, contiguity_df):
    """Get the list of neighboring precincts for a given precinct."""
    df = contiguity_df.copy()
    df.columns = ['Precinct', 'Neighbors']
    try:
        neighbors_str = df[df['Precinct'] == geoid]['Neighbors'].values[0]
        return ast.literal_eval(neighbors_str)
    except (KeyError, IndexError, ValueError):
        return []


total_nj_population = nj_precinct_data['Total_2020_Total'].sum()
avg_district_population = total_nj_population / 12
advisedSeedPopulation = avg_district_population * 0.10

def select_fair_seeds(precinct_data, boundaries, contiguity_df, num_districts=12):
    """
    Select seed precincts for a fair redistricting map.
    
    Strategy:
    1. Divide the state into num_districts roughly equal geographic regions
    1a. Because some districts will be "physically bigger in size" we will try to split the regions by population
    2. For each region, select a initial seed precinct that:
       - Is near the center of that region and contiguous
       - Has neighbors available for growth
    3. Flood fill up to advisedSeedPopulation
        - Population constraint of up to advisedSeedPopulation
        - contiguity constraint
        - Keep geographic constraint by adding precincts close to get_District_centroid()
        - Let's just initially try to focus on adding communities of interest for this seeding flood fill

    """
    
